In [6]:

from itertools import permutations
from joblib import Parallel, delayed
from more_itertools import distinct_permutations
from tqdm import tqdm
import numpy as np
import pickle

from qulacs import QuantumCircuit

from mcr.equiv_check import (
    equiv,
    pauli_bit_equivalence_check,
    equivalence_check_via_mqt_qcec,
)

from mcr.gate_apply import (
    PauliBit,
    grouping,
    loop_optimization,
    set_clifford_to_qulacs,
    zhang_optimization,
)

from opt_using_mcr import test_algorithm
from perform_mcr import (
    optimize_data_loop,
    attempt_mcr_retry,
    mcr_swap,
    three_layer_nontrivial_swap,
)
from optimizer import full_optimization

In [7]:
filetype = "seq"  # "small" or "seq"
nqubits = 2  # Number of qubits in the circuit
# with open(f"unopt_{filetype}.pickle", "rb") as f:
with open(f"unopt_{nqubits}.pickle", "rb") as f:
    seq = pickle.load(f)
data = []
for elem in seq:
    sgn = str(elem[1])[0]
    pauli_str = str(elem[1])[1:]
    if sgn == "+":
        data.append(PauliBit(pauli_str, np.pi/4))
    else:
        assert sgn == "-", f"Unexpected sign: {sgn}"
        data.append(PauliBit(pauli_str, -np.pi/4))
data.append(PauliBit("Z" * nqubits, -np.pi / 4))  # Add identity gate

In [8]:
def load_data_from_pickle(filepath: str, nqubits: int) -> list:
    """pickle形式のファイルを読み込む

    Args:
        filepath (str): pickle形式のファイルパス

    Returns:
        list: 読み込んだデータのリスト
    """
    with open(filepath, mode="br") as f:
        data = pickle.load(f)
    pauli_bit_sequences = []
    for elem in data:
        initial_pauli_str, theta, qubit_indices = elem
        pauli_str = ["I"] * nqubits
        for pauli, qubit_idx in zip(initial_pauli_str, qubit_indices):
            pauli_str[qubit_idx] = pauli
        pauli_bit = PauliBit("".join(pauli_str), theta)
        pauli_bit_sequences.append(pauli_bit)
    return pauli_bit_sequences
nqubits = 6
# data = load_data_from_pickle("te_pai_data.pickle", nqubits=nqubits)

In [9]:
clifford, non_clifford = full_optimization(data, max_iter = 10, show_opt_log=True)

🎉 Optimization success in iteration 1: 36 → 16
🔍 No optimization in iteration 2: 16 → 16
🔁 Additional optimization: 1 / 10
🎉 Optimization success in iteration 1: 23 → 14
🔍 No optimization in iteration 2: 14 → 14
🔁 Additional optimization: 2 / 10
🎉 Optimization success in iteration 1: 20 → 14
🔍 No optimization in iteration 2: 14 → 14
🔁 Additional optimization: 3 / 10
🎉 Optimization success in iteration 1: 20 → 14
🔍 No optimization in iteration 2: 14 → 14
🔁 Additional optimization: 4 / 10
🎉 Optimization success in iteration 1: 20 → 14
🔍 No optimization in iteration 2: 14 → 14
🔁 Additional optimization: 5 / 10
🎉 Optimization success in iteration 1: 20 → 14
🔍 No optimization in iteration 2: 14 → 14
🔁 Additional optimization: 6 / 10
🎉 Optimization success in iteration 1: 20 → 14
🔍 No optimization in iteration 2: 14 → 14
🔁 Additional optimization: 7 / 10
🎉 Optimization success in iteration 1: 20 → 14
🔍 No optimization in iteration 2: 14 → 14
🔁 Additional optimization: 8 / 10
🎉 Optimization s

In [10]:
non_clifford

[YZ, 0.785,
 YX, -0.785,
 ZZ, -0.785,
 YZ, 0.785,
 YX, -0.785,
 ZZ, -0.785,
 YZ, -0.785,
 ZX, 0.785,
 IY, -0.785,
 XI, 0.785,
 YX, -0.785,
 IY, -0.785,
 XI, -0.785,
 ZZ, -0.785]

In [11]:
grouping(non_clifford)

[[YZ, 0.785],
 [YX, -0.785, ZZ, -0.785],
 [YZ, 0.785],
 [YX, -0.785, ZZ, -0.785],
 [YZ, -0.785, ZX, 0.785],
 [IY, -0.785, XI, 0.785],
 [YX, -0.785],
 [IY, -0.785, XI, -0.785],
 [ZZ, -0.785]]

In [5]:
# itertools.permutations で全列挙は計算量が膨大なので、numpy配列でインデックスの順列を生成し、必要な部分だけ計算することで高速化します。
seq_a = [
    PauliBit("ix", -np.pi / 4),
    PauliBit("zi", -np.pi / 4),
    PauliBit("xy", np.pi / 4),
    PauliBit("zy", -np.pi / 4),
    PauliBit("yy", -np.pi / 4),
    PauliBit("zz", -np.pi / 4),
    PauliBit("ix", np.pi / 4),
    PauliBit("xi", np.pi / 4),
    PauliBit("yz", -np.pi / 4),
    PauliBit("xz", -np.pi / 4),
    PauliBit("zz", -np.pi / 4),
]
seq_b = [PauliBit("yx", -np.pi / 4)]
equiv([[], seq_a], [[],seq_b])

False

In [6]:
A = [
    PauliBit("IZ", -np.pi / 4),
    PauliBit("ZY", np.pi / 4),
    PauliBit("XY", -np.pi / 4),
    PauliBit("IZ", -np.pi / 4),
    PauliBit("XZ", -np.pi / 4),
    PauliBit("IY", -np.pi / 4),
    PauliBit("YZ", np.pi / 4),
    PauliBit("IX", -np.pi / 4),
    PauliBit("ZI", np.pi / 4),
    PauliBit("YY", -np.pi / 4),
    PauliBit("XI", -np.pi / 4),
    PauliBit("ZI", -np.pi / 4),
    PauliBit("ZZ", np.pi / 4),
    PauliBit("XI", np.pi / 4),
    PauliBit("ZX", np.pi / 4),
    PauliBit("XX", np.pi / 4),
]

In [7]:
m = grouping(A)
# m[8] += [PauliBit("XZ", -np.pi / 4)]
# m.insert(8, [PauliBit("XZ", np.pi / 4)])
m[8] += [PauliBit("XZ", np.pi / 4), PauliBit("XZ", -np.pi / 4)]
# m.insert(8, [PauliBit("XZ", -np.pi / 4)])
m

[[IZ, -0.785],
 [ZY, 0.785],
 [XY, -0.785],
 [IZ, -0.785, XZ, -0.785],
 [IY, -0.785],
 [YZ, 0.785],
 [IX, -0.785, ZI, 0.785],
 [YY, -0.785],
 [XI, -0.785, XZ, 0.785, XZ, -0.785],
 [ZI, -0.785, ZZ, 0.785],
 [XI, 0.785],
 [ZX, 0.785],
 [XX, 0.785]]

In [8]:
# ini = m.copy()
seq = mcr_swap(m)
clifford_lst, optimized_data = optimize_data_loop(
    seq, show_opt_log=True, max_attempts=3
)


Swapping [XI, -0.785, XZ, 0.785, XZ, -0.785] and [ZI, -0.785, ZZ, 0.785] at index 8
Swapping [XY, -0.785, YZ, 0.785] and [IX, -0.785, ZI, 0.785] at index 3
Swapping [XY, -0.785, YZ, 0.785] and [XZ, -0.785, YY, -0.785] at index 4
🎉 Optimization success in iteration 1: 16 → 8


In [9]:
clifford_lst, optimized_data = optimize_data_loop(
    seq, show_opt_log=True, max_attempts=3
)


Swapping [XY, -0.785, YZ, 0.785] and [IX, -0.785, ZI, 0.785] at index 3
Swapping [XY, -0.785, YZ, 0.785] and [XZ, -0.785, YY, -0.785] at index 4
🎉 Optimization success in iteration 1: 16 → 8


In [10]:
optimized_data

[]

In [11]:
three_layer_nontrivial_swap(grouping(A),with_mcr_index=True)

([IZ, -0.785,
  ZY, 0.785,
  IY, -0.785,
  IZ, -0.785,
  XZ, -0.785,
  XY, -0.785,
  YZ, 0.785,
  IX, -0.785,
  ZI, 0.785,
  YY, -0.785,
  XI, -0.785,
  ZI, -0.785,
  ZZ, 0.785,
  XI, 0.785,
  ZX, 0.785,
  XX, 0.785],
 {2})

In [12]:
p =grouping(optimize_data_loop(
    three_layer_nontrivial_swap(grouping(A)), show_opt_log=True
)[1])
p

🔍 No optimization in 1th iteration. 16 → 16
Swapping [XY, -0.785, YZ, 0.785] and [IX, -0.785, ZI, 0.785] at index 3
🔍 No optimization in 1th iteration. 16 → 16


[[IZ, -0.785],
 [IY, -0.785, ZY, 0.785],
 [IZ, -0.785, XZ, -0.785],
 [IX, -0.785, ZI, 0.785],
 [XY, -0.785, YZ, 0.785],
 [YY, -0.785],
 [XI, -0.785],
 [ZI, -0.785, ZZ, 0.785],
 [XI, 0.785],
 [ZX, 0.785],
 [XX, 0.785]]

In [13]:
seq_a = [
    PauliBit("ix", -np.pi / 4),
    PauliBit("xy", np.pi / 4),
    PauliBit("zy", -np.pi / 4),
    PauliBit("yy", -np.pi / 4),
    PauliBit("zz", -np.pi / 4),
    PauliBit("ix", np.pi / 4),
    PauliBit("xi", np.pi / 4),
    PauliBit("yz", -np.pi / 4),
    PauliBit("xz", -np.pi / 4),
    PauliBit("zz", -np.pi / 4),
]

# seq_b = [
#     PauliBit("XZ", np.pi / 4),
#     PauliBit("ZI", -np.pi / 4),
#     PauliBit("ZZ", np.pi / 4),
#     PauliBit("XZ", -np.pi / 4),
# ]
# equiv([[], seq_a], [[], seq_b])

In [14]:
m = grouping(seq_a)
m

[[IX, -0.785],
 [XY, 0.785],
 [ZY, -0.785],
 [YY, -0.785, ZZ, -0.785],
 [IX, 0.785, XI, 0.785],
 [YZ, -0.785],
 [XZ, -0.785],
 [ZZ, -0.785]]

In [15]:

m[2] += [PauliBit("YZ", np.pi / 4), PauliBit("YZ", -np.pi / 4)]
# optimize_data_loop(mcr_swap(m))
mcr_swap(m)

Swapping [ZY, -0.785, YZ, 0.785, YZ, -0.785] and [YY, -0.785, ZZ, -0.785] at index 2
Swapping [ZY, 0.785, YZ, 0.785] and [IX, 0.785, XI, 0.785] at index 3


[IX, -0.785,
 XY, 0.785,
 YZ, -0.785,
 ZY, -1.571,
 YY, -0.785,
 ZZ, -0.785,
 ZY, 1.571,
 IX, 0.785,
 XI, 0.785,
 ZY, -0.785,
 YZ, 0.785,
 YZ, -0.785,
 XZ, -0.785,
 ZZ, -0.785]